### Setup

In [ ]:
using Pkg

Pkg.add(url="https://github.com/mppmu/BoostFractor.jl.git")
Pkg.add(url="https://github.com/bergermann/Dragoon.jl.git")
Pkg.update()

In [ ]:
using Dragoon

In [ ]:
#number of discs in the booster
n = 20

#initial disc configuration
#findpeak tries to find an equidistant configuration with a peak at f
initdist = findpeak(22.025e9,n)

#generate frequencies for calculation and for plotting
freqs = genFreqs(22.025e9,50e6; length=10) #optimize on these frequencies
freqsplot = genFreqs(22.025e9,150e6; length=1000)

#initialize physical properties of the booster
booster = AnalyticalBooster(initdist)
#=
Booster(
    pos,              disc positions
    ndisk,            disc number
    thickness,        disc thickness
    epsilon,          disc epsilon
    vmotor,           motor speed
    maxlength,        maximum allowed booster length (not implemented yet)
    timestamp,        booster operation time
    summedtraveltime, summed motor operation time
    codetimestamp     code runtime
)
=#

#initialize storage of measurements
hist = initHist(booster,10000,freqs,ObjAnalytical)
#                           ^ length, should be at least the larger of (n^2, search steps)
;

In [ ]:
b1 = copy(booster.pos); display(b1)

In [ ]:
p = zeros(booster.ndisk)
g = zeros(booster.ndisk)
h = zeros(booster.ndisk,booster.ndisk)

updateHist!(booster,hist,freqs,ObjAnalytical)

deriv = Derivator2(1e-3,1e-3,"double")

secondDerivative(g,h,booster,hist,freqs,ObjAnalytical,(1e-3,1e-3,"double"))

b2 = copy(booster.pos); display(booster.pos)

In [ ]:
updateHist!(booster,hist,freqs,objFunction; force=true)

In [ ]:
booster.pos[1] = 1.
hist

In [ ]:
import Dragoon.move
args = (1e-3,1e-3,"double")
objFunction = ObjAnalytical

updateHist!(booster,hist,freqs,objFunction)
move(booster,[(1,args[1])])

if args[3] == "double"
    for i in 1:booster.ndisk
        updateHist!(booster,hist,freqs,objFunction)
        move(booster,[(i,-2*args[1])])
        updateHist!(booster,hist,freqs,objFunction)

        g[i] = (hist[2].objvalue-hist[1].objvalue)/(2*args[1])

        if i != booster.ndisk
            move(booster,[(i,args[1]),(i+1,args[1])])
        else
            move(booster,[(i,args[1])])
        end
    end

    for i in 1:booster.ndisk, j in 1:booster.ndisk
        move(booster,[(i,args[2]),(j,args[2])])
        updateHist!(booster,hist,freqs,objFunction)

        move(booster,[(j,-args[2])])
        updateHist!(booster,hist,freqs,objFunction)

        move(booster,[(i,-args[2]),(j,args[2])])
        updateHist!(booster,hist,freqs,objFunction)

        move(booster,[(j,-args[2])])
        updateHist!(booster,hist,freqs,objFunction)
        
        h[i,j] = (hist[4].objvalue-hist[3].objvalue-
                    hist[2].objvalue+hist[1].objvalue)/(args[2]^2)
    end
end

In [ ]:
updateHist!(booster,hist,freqs,objFunction; force=true)

In [ ]:
Dragoon.shiftdown!(hist)

In [ ]:
booster.pos[1] = 1.
hist

### Optimizer

In [ ]:
#=
how to use

linesearch(                     directly modifies booster.pos
    booster,                    
    hist,                       
    freqs,                      frequency range to optimize on
    α,                          base step length e.g. motorspeed*measurementfrequency
    (objective,[options]),      objective function
    (solver,[options]),         provides step direction p
    (derivative,[options]),     provides derivatives for use by solver
    (step,[options]),           calculates steplength
    (search,[options]),         search mode
    (unstuckinator,[options]);  tries to unstuck system from bad local minima
    ϵgrad=0,                    terminate if norm(gradient) falls below this value,
    maxiter=100,                maximum allowed iterations,
    showtrace=false,            wether to print progress,
    showevery=1,                only print every i iterations,
    unstuckisiter=true          wether an unstucking iteration counts towards maxiter
                                    warning! can run endlessly if false
)

returns trace, access with
trace[iteration].x      disc position
                .obj    objective value
                .g      gradient
                .h      hessian
                .t      timestamp
                .T      summed travel time
=#

In [ ]:
trace = linesearch(booster,hist,freqs,booster.vmotor*1e-3,
                    ObjAnalytical,
                    SolverNewton("cholesky"),
                    Derivator2(1e-5,1e-6,"double"),
                    StepNorm("unit"),
                    SearchExtendedSteps(2000),
                    UnstuckDont;
                    ϵgrad=0.,maxiter=Int(2e1),showtrace=true);

In [ ]:
#obtain result, booster is now in optimized position
#pos2dist(booster.pos)     #in distance space
booster.pos               #in position space

In [ ]:
#get handy output
plt = analyse(hist,trace,freqsplot; freqs=freqs,div=10)

#savefig(plt[i],"cool_epic_result_wow.svg")